In [8]:
import os 
from dotenv import load_dotenv

load_dotenv()

STORAGE_ACCOUNT_NAME = os.getenv("STORAGE_ACCOUNT")
CONTAINER_NAME = 'raw'
BLOB_PATH_ON_CONTAINER = 'tourism_dataset.csv'

%run ./read_data_from_az_storage.py \
    $STORAGE_ACCOUNT_NAME \
    $CONTAINER_NAME \
    $BLOB_PATH_ON_CONTAINER

     Location Country    Category  Visitors  Rating    Revenue  \
0  kuBZRkVsAR   India      Nature    948853    1.32   84388.38   
1  aHKUXhjzTo     USA  Historical    813627    2.01  802625.60   
2  dlrdYtJFTA  Brazil      Nature    508673    1.42  338777.11   
3  DxmlzdGkHK  Brazil  Historical    623329    1.09  295183.60   
4  WJCCQlepnz  France    Cultural    124867    1.43  547893.24   

  Accommodation_Available  
0                     Yes  
1                      No  
2                     Yes  
3                     Yes  
4                      No  


----
Query 1:
**Group and Aggregate Data:** 
Group the data by the 'country' column and calculate
the average value of the "Rate" column for each country.

Spark SQL Equivalent:

```sql
SELECT Country, AVG(Rating) as Rate
FROM csv.`/part_b/data/tourism_dataset.csv`
GROUP BY Country
```

Pandas:

In [9]:
q1 = df.groupby("Country")["Rating"].mean().reset_index()

q1

,Country,Rating
0,Australia,3.019602
1,Brazil,3.074167
2,China,2.958648
3,Egypt,3.024298
4,France,3.030268
5,India,2.973158
6,USA,2.984304


----
Query 2:
**Identify Top Categories:** 
Find the top 3 categories with the highest average rate
across all countries.

Spark SQL Equivalent:

```sql
WITH RankedCategories AS (
    SELECT 
        Country, 
        Category, 
        AVG(Rating) AS Rate, 
        ROW_NUMBER() OVER (PARTITION BY Country, Category ORDER BY AVG(Rating) DESC) AS RN
    FROM csv.`/part_b/data/tourism_dataset.csv`
    GROUP BY Country, Category
)

SELECT Country, Category, Rate
FROM RankedCategories
WHERE RN <= 3
```

Pandas:

In [10]:
q2 = df.groupby(["Country", "Category"])["Rating"].mean() \
    .reset_index() \
    .sort_values(by=["Country", "Rating"], ascending=[True, False]) \
    .groupby("Country").head(3)

q2

,Country,Category,Rating
1,Australia,Beach,3.229865
0,Australia,Adventure,3.080878
5,Australia,Urban,3.032578
8,Brazil,Cultural,3.254714
10,Brazil,Nature,3.180345
6,Brazil,Adventure,3.140563
13,China,Beach,3.069630
15,China,Historical,3.067111
17,China,Urban,2.957031
23,Egypt,Urban,3.109870


In [11]:
# Exporting and saving to blob. Will use blob uploader from part_a.

# Agg results 1
 
filepath_1 = "/home/andreas/Downloads/dss-main/part_b/export/Country-Avg-Rate-Andreas-Patsilivas.csv"
filename_1 = filepath_1.split("/")[-1]

q1.to_csv(filepath_1, index=False)

%run ../part_a/blob_uploader andreas-patsilivas \
    $filepath_1 \
    $filename_1

# Agg results 2

filepath_2 = "/home/andreas/Downloads/dss-main/part_b/export/Top3-Categories-Per-Country-Andreas-Patsilivas.csv"
filename_2 = filepath_2.split("/")[-1]

q2.to_csv(filepath_2, index=False)

%run ../part_a/blob_uploader andreas-patsilivas \
    $filepath_2 \
    $filename_2


Uploading file Country-Avg-Rate-Andreas-Patsilivas.csv to storage account: dataengineerv1, container name: andreas-patsilivas.
File uploaded to blob.
Uploading file Top3-Categories-Per-Country-Andreas-Patsilivas.csv to storage account: dataengineerv1, container name: andreas-patsilivas.
File uploaded to blob.
